In [41]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import *
from statsmodels.graphics.tsaplots import *
from catboost import *
from sklearn.model_selection import *
from sklearn.metrics import *
from sklearn.datasets import load_diabetes
from sklearn.model_selection import *
from sklearn.linear_model import *
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import *
from sklearn.ensemble import *
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.svm import *
from sklearn.neighbors import *
from sklearn.neural_network import *
from sklearn.metrics import *
import pickle
import tensorflow as tf
import shap
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import *
import os

plt.rcParams['font.sans-serif'] = ['SimHei']  # 黑体
plt.rcParams['axes.unicode_minus'] = False  # 解决无法显示符号的问题
palette = 'deep'
sns.set(font='SimHei', font_scale=1.2, style='whitegrid', palette=palette)  # 解决Seaborn中文显示问题

rawData = pd.read_excel('../../../原始数据/附件1：物流网络历史货量数据.xlsx')




In [67]:
filterSize = 2
def getxy():
    targetKey = '货量'
    # 对场地进行编码

    df = rawData.copy()
    # df = df[(df['场地1'] == 'DC14') & (df['场地2'] == 'DC10')].reset_index(drop=True)
    df['场地1'] = df['场地1'].str.replace('DC', '')
    df['场地1'] = df['场地1'].astype('int64')
    df['场地2'] = df['场地2'].str.replace('DC', '')
    df['场地2'] = df['场地2'].astype('int64')

    df['日期'] = pd.to_datetime(df['日期'])
    df['日期'] = df['日期'] - df['日期'].min()
    df['日期'] = df['日期'].apply(lambda x: x.days)

    df['货量'] = np.log(df['货量'])
    # return df.drop(targetKey, axis=1), df[targetKey], df

    grouped = df.groupby(['场地1', '场地2']).size().reset_index(name='count')
    merged = pd.merge(df, grouped, on=['场地1', '场地2'], how='left').reset_index(drop=True)
    ltSize = merged[merged['count'] < filterSize]
    print(f'总行数有：{df.shape}')
    print(f'数据小于{filterSize}的行数有：{ltSize.shape}')
    filtered = merged[merged['count'] >= filterSize]
    filtered = filtered.drop('count', axis=1)
    print(f'清洗后数量：{filtered.shape}')
    return filtered.drop(targetKey, axis=1), filtered[targetKey], filtered, ltSize, df

In [68]:
_, _, filtered, ltSize, df = getxy()

总行数有：(177847, 4)
数据小于2的行数有：(121, 5)
清洗后数量：(177726, 4)


In [69]:
ltSize

,场地1,场地2,日期,货量,count
2332,43,45,9,1.386294,1
4155,7,34,17,1.098612,1
4624,58,29,18,0.000000,1
4660,9,22,19,1.609438,1
5207,14,67,21,0.693147,1
...,...,...,...,...,...
171718,33,65,711,0.000000,1
172254,65,43,712,1.098612,1
174015,10,59,718,3.465736,1
174469,32,10,719,1.609438,1


In [70]:
print(ltSize.groupby(['场地1', '场地2']).size())

场地1  场地2
1    8      1
3    19     1
     28     1
     30     1
     38     1
           ..
70   34     1
73   20     1
75   35     1
76   8      1
77   6      1
Length: 121, dtype: int64
